In [6]:
#!pip install google-generativeai

In [7]:
import google.generativeai as genai
from google.colab import userdata

In [8]:
#Set the API Key
genai.configure(api_key = userdata.get("GOOGLE_API_KEY"))

#1. Text model

In [9]:
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("who are the prime ministers or Malaysia")
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Here is a list of Prime Ministers of Malaysia, starting from the country's independence in 1957:\n\n**1. Tunku Abdul Rahman Putra Al-Haj (1957-1970)** -  First Prime Minister, known for leading Malaysia to independence from British rule.\n\n**2. Tun Abdul Razak Hussein (1970-1976)** -  Second Prime Minister, instrumental in developing the nation's economy and fostering national unity.\n\n**3. Tun Hussein Onn (1976-1981)** -  Third Prime Minister, continued the economic development policies of his predecessor.\n\n**4. Tun Dr. Mahathir Mohamad (1981-2003 & 2018-2020)** -  Fourth and Seventh Prime Minister, a controversial figure known for his economic reforms and bold policies. He served two separate terms.\n\n**5. Datuk Seri Abdullah Ahmad Badawi (2003-2009)*

In [10]:
print(response.text)

Here is a list of Prime Ministers of Malaysia, starting from the country's independence in 1957:

**1. Tunku Abdul Rahman Putra Al-Haj (1957-1970)** -  First Prime Minister, known for leading Malaysia to independence from British rule.

**2. Tun Abdul Razak Hussein (1970-1976)** -  Second Prime Minister, instrumental in developing the nation's economy and fostering national unity.

**3. Tun Hussein Onn (1976-1981)** -  Third Prime Minister, continued the economic development policies of his predecessor.

**4. Tun Dr. Mahathir Mohamad (1981-2003 & 2018-2020)** -  Fourth and Seventh Prime Minister, a controversial figure known for his economic reforms and bold policies. He served two separate terms.

**5. Datuk Seri Abdullah Ahmad Badawi (2003-2009)** -  Fifth Prime Minister, faced criticism for his handling of the economy and corruption issues.

**6. Datuk Seri Najib Razak (2009-2018)** -  Sixth Prime Minister, involved in a major corruption scandal known as 1MDB, leading to his eventua

In [11]:
#Text model configuration
response = model.generate_content("Who are the prime ministers of Malaysia",
                                  generation_config = genai.types.GenerationConfig(
                                      max_output_tokens=200, #Max tokens
                                      stop_sequences=[','], #Stop sequence
                                      temperature=2
                                  )
                                )
print(response.text)

Malaysia has had **10 prime ministers** since its independence in 1957. Here's a list:

1. **Tunku Abdul Rahman Putra Al-Haj** (1957 - 1970) - Founder of modern Malaysia


In [12]:
#Checking usage
print(response.usage_metadata)

prompt_token_count: 8
candidates_token_count: 55
total_token_count: 63



In [13]:
#You can create multiple models with different instructions
teacher_model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction = """
                              You are a school teacher and will elaborate on mathematics.
                              You refuse to respond to anything related to sports""")

coach_model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction = """
                              You are a school coach and will elaborate on physical education.
                              You refuse to respond to anything related to mathematics""")

In [14]:
response = coach_model.generate_content("How to do push ups")
print(response.text)

Alright, team, let's talk about push-ups! They're a great exercise to build upper body strength and endurance. Here's the breakdown:

**Starting Position:**

1. **Hands Shoulder-Width Apart:**  Place your hands flat on the ground, slightly wider than shoulder-width apart. Your fingers should be pointing forward.
2. **Body Straight:**  Keep your body in a straight line from your head to your heels.  
3. **Engaged Core:**  Tighten your stomach muscles to keep your body stable.

**The Movement:**

1. **Lower Down:** Bend your elbows and lower your chest towards the ground.  Keep your elbows close to your body. 
2. **Push Up:**  Push back up to the starting position, extending your arms fully. 

**Key Points:**

* **Focus on Form:**  Don't rush the movement. Maintain a straight line throughout the exercise.
* **Control the Descent:**  Lower yourself slowly and with control.
* **Full Extension:**  Extend your arms completely at the top of the movement.
* **Engage Your Core:**  Keep your cor

2. Vision model

In [15]:
#Import to open image from file path
import PIL.Image

#Open image
image = PIL.Image.open("/content/bagpack_cat.png")

#Using Gemini API for vision on a single image
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(["describe this image.",image])
print(response.text)

FileNotFoundError: [Errno 2] No such file or directory: '/content/bagpack_cat.png'

In [ ]:
#Using Gemini API for vision on multiple images
image1 = PIL.Image.open("/content/bagpack_cat.png")
image2 = PIL.Image.open("/content/tiger.jpeg")

model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(["Compare these images.",image1,image2])
print(response.text)

In [ ]:
#Gemini API does not have access to websites
image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTmbc4YToclOg_z75POxniY4JLc36V_0dPLXA&s"
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(["Describe this image.",image_url])
print(response.text)

# 3. Recipe Generator using Gemini API Vision Model

- Generate a recipe from an image of ingredients and format the output to upscale the ingredients

In [ ]:
#Initialise the model
recipe_vision = genai.GenerativeModel(
    "gemini-1.5-flash",
    system_instruction="""
    You are a chef trying to assist normal homecooks with cooking at home.
    You will be given an image of ingredients and generate a recipe based on
    the ingredients for a single serving.
    The output must be in JSON format. Do not mention json.
    Output contains
    1. recipe name
    2. ingredients: quantity, unit, name
    3. Instructions
    """)

#Use your own image
image = PIL.Image.open("/content/produce.jpg")
response = recipe_vision.generate_content(['Here are the ingredients:',image])

In [ ]:
#Observe output characteristics
print(response.text)

In [ ]:
#Load from json format into Python dictionary
import json
recipe = json.loads(response.text)

In [ ]:
#Required to convert Fraction into float
from fractions import Fraction

#Observe your output and adjust accordingly
def display_recipe(pax, recipe):
  #Print Recipe Name
  print(f"{recipe['recipe_name']}")
  print('-'*50)

  #Print Ingredients
  for ingredient in recipe['ingredients']:
    #Scale up ingredients
    quantity = float(Fraction(ingredient['quantity']))*pax
    print(f"{quantity} {ingredient['unit']} {ingredient['name']}")
  print("-"*50)

  #Print Instructions
  for instruction in recipe['instructions']:
    print(instruction)

In [ ]:
#num of pax
pax = input("Enter num of pax")
display_recipe(int(pax),recipe)

In [ ]:
#FYI. You can use an AI model to do the work for you instead of doing it manually ;)